<a href="https://colab.research.google.com/github/lukman99-web/Analisis-Big-DATA/blob/main/add_subtitle_english.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

# Install library yang dibutuhkan
!pip install openai-whisper
!pip install moviepy
!pip install deep-translator
!pip install ffmpeg-python

# Install ffmpeg (jika belum ada di environment Colab)
!apt-get install ffmpeg -qq

In [4]:
# @title Make Subtitle

import os
import whisper
from deep_translator import GoogleTranslator
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip
import datetime
from google.colab import files

# 1. Pilih Bahasa Output
print("Pilih bahasa terjemahan:")
print("1. Bahasa Inggris (English)")
print("2. Bahasa Arab (Arabic)")
choice = input("Masukkan pilihan (1 atau 2): ")

if choice == '1':
    target_lang = 'en'
    lang_name = 'English'
elif choice == '2':
    target_lang = 'ar'
    lang_name = 'Arabic'
else:
    print("Pilihan tidak valid, default ke Bahasa Inggris.")
    target_lang = 'en'
    lang_name = 'English'

# 2. Upload Video
print("\nSilakan upload file video Anda...")
uploaded = files.upload()

# Ambil nama file yang diupload
video_filename = list(uploaded.keys())[0]
print(f"File diterima: {video_filename}")

# 3. Load Model Whisper dan Transkripsi
print("\nMemproses audio (Transkripsi)... Ini mungkin memakan waktu.")
# Menggunakan model 'base' untuk keseimbangan kecepatan dan akurasi. Ganti ke 'small' atau 'medium' jika butuh lebih akurat.
model = whisper.load_model("base")
result = model.transcribe(video_filename)

# 4. Terjemahan Teks
print(f"Menerjemahkan teks ke {lang_name}...")
translator = GoogleTranslator(source='auto', target=target_lang)

# Fungsi untuk membuat format waktu SRT (SubRip)
def format_timestamp(seconds):
    td = datetime.timedelta(seconds=seconds)
    total_seconds = int(td.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)
    minutes, secs = divmod(remainder, 60)
    millis = int((seconds - int(seconds)) * 1000)
    return f"{hours:02}:{minutes:02}:{secs:02},{millis:03}"

# Menyimpan subtitle ke file .srt
srt_filename = "subtitle_output.srt"
with open(srt_filename, "w", encoding="utf-8") as f:
    for i, segment in enumerate(result['segments']):
        start_time = format_timestamp(segment['start'])
        end_time = format_timestamp(segment['end'])

        # Terjemahkan teks per segmen
        original_text = segment['text'].strip()
        translated_text = translator.translate(original_text)

        f.write(f"{i+1}\n")
        f.write(f"{start_time} --> {end_time}\n")
        f.write(f"{translated_text}\n\n")

print(f"File subtitle '{srt_filename}' berhasil dibuat.")

# 5. Tanamkan Subtitle ke Video (Hardsub)
# Menggunakan ffmpeg untuk menempelkan subtitle ke video
output_video = f"translated_output.mp4"

print("\nMemproses video final (Menambahkan subtitle)...")
# Font untuk Arab memerlukan font yang mendukung karakter Arab. Colab biasanya punya DejaVu Sans.
# Jika bahasa Arab, kita gunakan font yang kompatibel.
font_setting = "DejaVu-Sans"
if target_lang == 'ar':
    font_setting = "DejaVu-Sans" # Font yang mendukung Arab di Linux/Colab

# Command ffmpeg untuk burn subtitle
# vf_subtitle = f"subtitles={srt_filename}:force_style='FontName={font_setting},FontSize=24,PrimaryColour=&HFFFFFF'"
command = f'ffmpeg -i "{video_filename}" -vf "subtitles={srt_filename}:force_style=\'FontName={font_setting},FontSize=18\'" -c:a copy "{output_video}" -y'

# Jalankan command
os.system(command)

print(f"\nProses selesai! File output: {output_video}")

# 6. Download Hasil
files.download(output_video)

Pilih bahasa terjemahan:
1. Bahasa Inggris (English)
2. Bahasa Arab (Arabic)
Masukkan pilihan (1 atau 2): 1

Silakan upload file video Anda...


Saving semusim.mp4 to semusim.mp4
File diterima: semusim.mp4

Memproses audio (Transkripsi)... Ini mungkin memakan waktu.


  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



Menerjemahkan teks ke English...
File subtitle 'subtitle_output.srt' berhasil dibuat.

Memproses video final (Menambahkan subtitle)...

Proses selesai! File output: translated_output.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>